In [ ]:
#from main import *
from tqdm import tqdm
import networkx as nx
from pyecharts.options.global_options import InitOpts
import scipy.io as sio
import h5py
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
import pyecharts
import random
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType, GeoType
from shapely.wkb import dumps as b_dumps, loads as b_loads
import os
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import statsmodels.api as sm # recommended import according to the docs
import copy
from multiprocessing import  Process


'''Notice, modify the constellation params if needed'''

In [ ]:
RADIUS = 6371000
# constellation parameters
constell_num = 2
Constellation = [0]*constell_num
HEIGHT = [0]*constell_num
ELEVATION = [0]*constell_num
NUM_ORBITS = [0]*constell_num
NUM_SATS_PER_ORBIT = [0]*constell_num
ANUM = [0]*constell_num
CONSTELL_ID = 0
Constellation[CONSTELL_ID] = "Starlink1" 
HEIGHT[CONSTELL_ID] = 550000
ELEVATION[CONSTELL_ID] = 25
NUM_ORBITS[CONSTELL_ID] = 72
NUM_SATS_PER_ORBIT[CONSTELL_ID] = 22
ANUM[CONSTELL_ID] = 2  
# CONSTELL_ID = 0
# Constellation[CONSTELL_ID] = 'Kuiper1'
# HEIGHT[CONSTELL_ID] = 630000
# ELEVATION[CONSTELL_ID] = 20
# NUM_ORBITS[CONSTELL_ID] = 34
# NUM_SATS_PER_ORBIT[CONSTELL_ID] = 34
# ANUM[CONSTELL_ID] = 2

constell_id=0#starlink 1584
NUM_SATELLITES = NUM_ORBITS[constell_id] * NUM_SATS_PER_ORBIT[constell_id]
satStartId = [0]
orbit_num=NUM_ORBITS[CONSTELL_ID]#72
sat_per_orbit=NUM_SATS_PER_ORBIT[CONSTELL_ID]#22
sat_num=orbit_num*sat_per_orbit
fp = "../data/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp"
data = gpd.read_file(fp)
def cir_to_car(lat, lng, h):
    return (RADIUS+h)*math.cos(math.radians(lat))*math.cos(math.radians(lng)),\
           (RADIUS+h)*math.cos(math.radians(lat))*math.sin(math.radians(lng)),\
           (RADIUS+h)*math.sin(math.radians(lat))

def dis_car(A, B):
    dis2 = (A[0]-B[0])*(A[0]-B[0])+(A[1]-B[1])*(A[1]-B[1])+(A[2]-B[2])*(A[2]-B[2])
    return math.sqrt(dis2)

def elevation_from_dis(adis, h_sat):
    if(h_sat >= adis):
        ael = 90
    else:
        cosel = (RADIUS**2 + adis**2 - (RADIUS+h_sat)**2) / (2*RADIUS*adis) 
        ael = math.degrees(math.acos(cosel)) - 90
    return ael
def get_arc(p0, p1, r=6371000+HEIGHT[CONSTELL_ID]):#550000): # p0=[lon,lat]
	z0 = r*np.sin(np.radians(p0[1]))
	x0 = r*np.cos(np.radians(p0[1]))*np.cos(np.radians(p0[0]))
	y0 = r*np.cos(np.radians(p0[1]))*np.sin(np.radians(p0[0]))
	z1 = r*np.sin(np.radians(p1[1]))
	x1 = r*np.cos(np.radians(p1[1]))*np.cos(np.radians(p1[0]))
	y1 = r*np.cos(np.radians(p1[1]))*np.sin(np.radians(p1[0]))
	theta = np.arccos(np.dot((x0,y0,z0),(x1,y1,z1))/(r*r))
	return theta * r
def cycleCompute(height):
    r = RADIUS + height
    G = 6.67428e-11
    M = 5.965e+24
    res = 4*math.pi*math.pi*r*r*r/G
    res = res/M
    res = math.sqrt(res)
    return res


In [ ]:
LIGHTSPEED=299792458
FIBERSPEED = LIGHTSPEED * 2 / 3
lasting=86400
import concurrent.futures
import multiprocessing
import csv
from threading import Thread
def load_topo(satPos):
    GRAPH_ISL=nx.Graph()
    i=0
    for orbitId in range(orbit_num):
        for satInOrbitId in range(sat_per_orbit):
            satId = orbitId * sat_per_orbit + satInOrbitId + satStartId[i] # satID = pre sat + inner id
            #GRAPH_ISL.add_node(satId)
            neighborOrbitId = orbitId
            neighborSatInOrbitId = (satInOrbitId + 1) % sat_per_orbit
            neighborSatId = neighborOrbitId * sat_per_orbit + neighborSatInOrbitId + satStartId[i]
            dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
            GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
            # down
            neighborOrbitId = orbitId
            neighborSatInOrbitId = (satInOrbitId - 1 + sat_per_orbit) % sat_per_orbit
            neighborSatId = neighborOrbitId * sat_per_orbit + neighborSatInOrbitId + satStartId[i]
            dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
            GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
            # left
            neighborOrbitId = (orbitId - 1 + orbit_num) % orbit_num
            neighborSatInOrbitId = satInOrbitId
            neighborSatId = neighborOrbitId *sat_per_orbit + neighborSatInOrbitId + satStartId[i]
            dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
            GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
            # right
            neighborOrbitId = (orbitId + 1) % orbit_num
            neighborSatInOrbitId = satInOrbitId
            neighborSatId = neighborOrbitId *sat_per_orbit + neighborSatInOrbitId + satStartId[i]
            dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
            GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
    return GRAPH_ISL
def load_sd_sats(sd_no,t,constellation_name='Starlink1'):
    if constellation_name=='Starlink1':
        file_nx=open(f'/mnt/data/research3/tmp_chenwei_disk/path_validation/sd_sats_24h/{t}.txt','r')
    else:
        file_nx=open(f'/data/chenwei/research3/data/sd_sats_topo/Kuiper1/{t}.txt','r')
    lines1=file_nx.readlines()
    sd_now=eval(lines1[sd_no])
    file_nx.close()
    return sd_now
def rh(ps,pd):
    #horizontal path, right=1,left=0
    right=abs(pd-ps)
    left=orbit_num-right
    if right>left:return 1
    return 0
# def rv(ns,nd,d):
# vertical path, up=1, down=0, differentiate directions, only applied to the two sats in the same direction
#     if d=='N':
#         if ns>nd:
#         up=abs(nd-ns)
        
def load_nlrp(t,risk_country):
    if Constellation[CONSTELL_ID]=='Starlink1':
        file=f'/data/chenwei/research3/data/NLRP_24h/{risk_country}/{t}.txt'
    else:
        file=f'/data/chenwei/research3/data/NLRP_24h/Kuiper1/{risk_country}/{t}.txt'
    f=open(file,'r')
    lines=f.readlines()
    nlrp=eval(lines[0])
    rsats=eval(lines[1])
    rs=[]
    for items in rsats:
        rs.extend(list(items))
    f.close()
    return nlrp,rs
def is_risk(nlrp,path,d):
    l,r,u,b=nlrp[0],nlrp[1],nlrp[2],nlrp[3]
    rsats=[]
    if d=='N':
        if u>b and l<r:
            for p in range(l+1,r):
                for n in range(b+1,u):
                    rsats.append(get_sid(p,n))
        elif u>b and l>r:
            if l+1>orbit_num:l=1
            else:l+=1
            for n in range(b+1,u):
                for p in range(l,orbit_num+1):
                    rsats.append(get_sid(p,n))
                for p in range(1,r):
                    rsats.append(get_sid(p,n))
        elif u<b and l<r:
            if b+1>sat_per_orbit:b=1
            else:b+=1
            for p in range(l+1,r):
                for n in range(b,sat_per_orbit+1):
                    rsats.append(get_sid(p,n))
                for n in range(1,u):
                    rsats.append(get_sid(p,n))
        else:
            if l+1>orbit_num:l=1
            else:l+=1
            if b+1>sat_per_orbit:b=1
            else:b+=1
            for p in range(l,orbit_num+1):
                for n in range(b,sat_per_orbit+1):
                    rsats.append(get_sid(p,n))
                for n in range(1,u):
                    rsats.append(get_sid(p,n))
            for p in range(1,r):
                for n in range(b,sat_per_orbit+1):
                    rsats.append(get_sid(p,n))
                for n in range(1,u):
                    rsats.append(get_sid(p,n))
    else:
        if u>b and l<r:
            if u+1>sat_per_orbit:u=1
            else:u+=1
            for p in range(l+1,r):
                for n in range(u,sat_per_num+1):
                    rsats.append(get_sid(p,n))
                for n in range(1,b):
                    rsats.append(get_sid(p,n))
        elif u>b and l>r:
            if l+1>orbit_num:l=1
            else:l+=1
            if u+1>sat_per_orbit:u=1
            else:u+=1 
            for p in range(l,orbit_num+1):
                for n in range(u,sat_per_orbit+1):
                    rsats.append(get_sid(p,n))
                for n in range(1,b):
                    rsats.append(get_sid(p,n))
            for p in range(1,r):
                for n in range(u,sat_per_orbit+1):
                    rsats.append(get_sid(p,n))
                for n in range(1,b):
                    rsats.append(get_sid(p,n))
        elif u<b and l<r:
            for p in range(l+1,r):
                for n in range(u+1,b):
                    rsats.append(get_sid(p,n))
        else:
            if l+1>orbit_num:l=1
            else:l+=1
            for n in range(u+1,b):
                for p in range(l,orbit_num+1):
                    rsats.append(get_sid(p,n))
                for p in range(1,r):
                    rsats.append(get_sid(p,n))
    for node in path:
        if node in rsats:
            return 1
    return 0
def optplane(pf,nlrp,loc0,loc1):
    if len(nlrp)==0:return -1
    l,r,u,b=nlrp[0],nlrp[1],nlrp[2],nlrp[3]
    if pf==0:
        hl=min(abs(loc0[0]-l),orbit_num-abs(loc0[0]-l))+min(abs(loc1[0]-l),orbit_num-abs(loc1[0]-l))
        hr=min(abs(loc0[0]-r),orbit_num-abs(loc0[0]-r))+min(abs(loc1[0]-r),orbit_num-abs(loc1[0]-r))
        return l if hl<hr else r
    else:
        vu=min(abs(loc0[1]-u),sat_per_orbit-abs(loc0[1]-u))+min(abs(loc1[1]-u),sat_per_orbit-abs(loc1[1]-u))
        vb=min(abs(loc0[1]-b),sat_per_orbit-abs(loc0[1]-b))+min(abs(loc1[1]-b),sat_per_orbit-abs(loc1[1]-b))
        return u if vu<vb else b
def init_log(lasting):
#     error=[[0,0] for _ in range(lasting)]#[fn,fp]
#     relays=[[]for _ in range(lasting)]
#     lantency=[0 for _ in range(lasting)]
    info=[[[],0,0,0,0]for _ in range(lasting)]
    return info
def load_topo_opt(rsats,satPos):
    GRAPH_ISL=nx.Graph()
    i=0
    for orbitId in range(orbit_num):
        for satInOrbitId in range(sat_per_orbit):
            satId = orbitId * sat_per_orbit + satInOrbitId + satStartId[i] # satID = pre sat + inner id
            if satId not in rsats:
                #GRAPH_ISL.add_node(satId)
                neighborOrbitId = orbitId
                neighborSatInOrbitId = (satInOrbitId + 1) % sat_per_orbit
                neighborSatId  = neighborOrbitId * sat_per_orbit + neighborSatInOrbitId + satStartId[i]
                if neighborSatId not in rsats:
                    dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
                    GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
                # down
                neighborOrbitId = orbitId
                neighborSatInOrbitId = (satInOrbitId - 1 + sat_per_orbit) % sat_per_orbit
                neighborSatId = neighborOrbitId * sat_per_orbit + neighborSatInOrbitId + satStartId[i]
                if neighborSatId not in rsats:
                    dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
                    GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
                # left
                neighborOrbitId = (orbitId - 1 + orbit_num) % orbit_num
                neighborSatInOrbitId = satInOrbitId
                neighborSatId = neighborOrbitId *sat_per_orbit + neighborSatInOrbitId + satStartId[i]
                if neighborSatId not in rsats:
                    dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
                    GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
                # right
                neighborOrbitId = (orbitId + 1) % orbit_num
                neighborSatInOrbitId = satInOrbitId
                neighborSatId = neighborOrbitId *sat_per_orbit + neighborSatInOrbitId + satStartId[i]
                if neighborSatId not in rsats:
                    dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])
                    GRAPH_ISL.add_edge(satId, neighborSatId,hop=1,weight=dis/LIGHTSPEED)
    return GRAPH_ISL
def is_in_risk_area(rsats,path):
    for n in path:
        if n in rsats:
            return n
    return -1
def p_in_NLRP(nlrp,p,d=1,enlarge=0):#default direction is N
    if nlrp[1-d]==[]:return 0
    d=1-d
    l,r=nlrp[d][enlarge][0],nlrp[d][enlarge][1]
    if l<r:
        if p>l and p<r:return 1
    else:
        if (p>l and p<=orbit_num) or (p>=1 and p<r):return 1
    return 0
def n_in_NLRP(nlrp,n,d=1,enlarge=0):
    if nlrp[1-d]==[]:return 0
    d=1-d
    u,b=nlrp[d][enlarge][2],nlrp[d][enlarge][3]
    if 1-d==1:
        if u>b:
            if u>n and n>b:return 1
        else:
            if (n>b and n<=sat_per_orbit) or (n>=1 and n<u):return 1
    else:
        if u<b:
            if n>u and n<b:return 1
        else:
            if (n>u and n<=sat_per_orbit) or (n>=1 and n<b):return 1
    return 0
def DRSA(sat0,sat1,ds,dd,nlrp,enlarge):
    ps,ns=get_satno(sat0)
    pd,nd=get_satno(sat1)
    rs=[]
    if ds==dd:
        si,di=n_in_NLRP(nlrp,ns,ds,enlarge),n_in_NLRP(nlrp,nd,ds,enlarge)
        if si and di:#only North Korea and Egypt
#             if p_in_NLRP(nlrp,ps,ds,enlarge) or p_in_NLRP(nlrp,pd,ds,enlarge):
#                 rs.append(get_sid(ps,nd))
#             else:
            rs.extend([get_sid(ps,nlrp[1-ds][enlarge][3]),get_sid(pd,nlrp[1-ds][enlarge][3])])
                
            
        elif not si and di:
            rs.append(get_sid(pd,ns))
        elif si and not di:
            rs.append(get_sid(ps,nd))
        else:
            rs.append(get_sid(ps,nd))
    else:
        sN,dN=p_in_NLRP(nlrp,ps,1,enlarge),p_in_NLRP(nlrp,pd,1,enlarge)
        sS,dS=p_in_NLRP(nlrp,ps,0,enlarge),p_in_NLRP(nlrp,pd,0,enlarge)
        if sN|sS and not dN|dS:
            rs.append(get_sid(pd,ns))
        elif dN|dS and not sN|sS:
            rs.append(get_sid(ps,nd))
        elif not dN|dS and not sN|sS:
            if ds:
                if len(nlrp[0])==0:rs.append(get_sid(ps,nd))
                else:
                    rs.extend([get_sid(ps,nlrp[0][enlarge][3]),get_sid(pd,nlrp[0][enlarge][3])])
            else:
                if len(nlrp[1])==0:rs.append(get_sid(pd,ns))
                else:
                    rs.extend([get_sid(ps,nlrp[1][enlarge][3]),get_sid(pd,nlrp[1][enlarge][3])])
#             snN,dnN=n_in_NLRP(nlrp,ns,1,enlarge),n_in_NLRP(nlrp,nd,1,enlarge)
#             snS,dnS=n_in_NLRP(nlrp,ns,0,enlarge),n_in_NLRP(nlrp,nd,0,enlarge)
#             if snN and dnN:
#                 rs.extend([get_sid(ps,nlrp[0][enlarge][3]),get_sid(pd,nlrp[0][enlarge][3])])
#             elif snN and not dnN:
#                 if dnS
#                 rs.append(get_sid(ps,nd))
#             elif not snN and dnN:
#                 rs.append(get_sid(pd,ns))
            
#             if not n_in_NLRP(nlrp,ns,1,enlarge)and not n_in_NLRP(nlrp,ns,1,enlarge)
#             rs.append(get_sid(ps,min(min(ns,nd),nlrp[0][enlarge][3])))
        else:
            if sN and dN:
                plane=optplane(0,nlrp[0][enlarge],[ps,ns],[pd,nd])
 
                rs.extend([get_sid(plane,ns),get_sid(plane,nd)])
       
            elif sN and not dN:
                try:
                    if ds:#s为北
                        plane=optplane(0,nlrp[0][enlarge],[ps,ns],[pd,nd])
                        rs.extend([get_sid(plane,ns),get_sid(plane,nd)])
                        
                    else:
                        plane=optplane(0,nlrp[1][enlarge],[ps,ns],[pd,nd])
                        if plane!=-1:
                            rs.extend([get_sid(plane,ns),get_sid(plane,nd)])
                        else:
                            rs.append(get_sid(pd,ns))
                except:
                    rs.append(get_sid(pd,ns))
            elif not sN and dN:
                try:
                    
                    plane=optplane(0,nlrp[1-ds][enlarge],[ps,ns],[pd,nd])
                    if plane!=-1:
                        rs.extend([get_sid(plane,ns),get_sid(plane,nd)])
                    else:
                        rs.append(get_sid(ps,nd))
                except:
                    rs.append(get_sid(ps,nd))
            else:
                try:
                    plane=optplane(0,nlrp[1][enlarge],[ps,ns],[pd,nd])
                    if plane!=-1:
                        rs.extend([get_sid(plane,ns),get_sid(plane,nd)])
                    else:
                        rs.append(get_sid(ps,nd))
                except:
                    rs.append(get_sid(ps,nd))
#                 if sS:
#                     rs.extend([get_sid(nlrp[1][enlarge][0],ns),get_sid(nlrp[1][enlarge][0],nd)])
#                 else:
#                     rs.append(ps,nd)
    if rs!=[] and rs[0]==sat0:rs.remove(sat0)
    if rs!=[] and rs[-1]==sat1:rs.remove(sat1)
    return rs
def starveri_run(sd_no,risk_country_ls,lasting,thread_num,constellation_name='Starlink1'):
    results0=[[]for _ in range(lasting)]
    results1=[[]for _ in range(lasting)]
    def starveri_main(sd_no,tstart,t,risk_country_ls,sd_pre,rsats_pre=[],nlrp_pre=[]):
        satfile=f'/home/chenwei/CrossShellRouting/pos_data/{constellation_name}/{constellation_name}_{t}.txt'
        satPos=load_satPos(satfile)
        sd=load_sd_sats(sd_no,t,constellation_name)
        rc0=risk_country_ls[0]
        nlrp0,rsats0=load_nlrp(t,rc0)
        f1,f2=0,0
        if t!=tstart and (sd==sd_pre and nlrp_pre[0]==nlrp0):
            f1=1
            results0[t]=results0[t-1]
        rc1=risk_country_ls[1]
        nlrp1,rsats1=load_nlrp(t,rc1)
        
        if t!=tstart and (sd==sd_pre and nlrp_pre[1]==nlrp1):
            f2=1
            results1[t]=results1[t-1]

        cflag1=0
        if sd[0] in rsats0 or sd[1] in rsats0:
            results0[t]=[[],[],[]]
            cflag1=1

        cflag2=0
        if sd[0] in rsats1 or sd[1] in rsats1:
            results1[t]=[[],[],[]]
            cflag2=1
        
        ds,dd=get_direction(satPos,t,sd[0],lasting,constellation_name),get_direction(satPos,t,sd[1],lasting,constellation_name)
        
        graph=load_topo(satPos)
        
#         all_delay=dict(nx.all_pairs_dijkstra_path_length(graph))
#         all_path=dict(nx.all_pairs_dijkstra_path(graph))
        #1risk country
        try:
            if (not f1) and (not cflag1):
                nlrp,rsats=nlrp0,rsats0
                graph_opt=load_topo_opt(rsats,satPos)
                min_delay=nx.dijkstra_path_length(graph_opt,sd[0],sd[1])
                threshold=[min_delay]
                for enlarge in range(3):
                    delay=0
                    tr=[]
                    rs=[sd[0]]
                    rs.extend(DRSA(sd[0],sd[1],ds,dd,nlrp,enlarge))
                    rs.append(sd[1])

                    for ind in range(len(rs)-1):
                        D=nx.dijkstra_path_length(graph,rs[ind],rs[ind+1])
                        delay+=D#all_delay[rs[ind]][rs[ind+1]]
                        min_seg_thresh=10000
                        rnode=-1
                        if len(rsats)==0:
                            tr.append([])
                        else:
                            path=nx.dijkstra_path(graph,rs[ind],rs[ind+1])
                            for node in path:#nx.dijkstraall_path[rs[ind]][rs[ind+1]]:
                                for rsid in rsats:
                                    delays=nx.dijkstra_path_length(graph,rsid,node)#all_delay[rsid][node]
                                    if delays<min_seg_thresh:
                                        rnode=rsid
                                        min_seg_thresh=delays
                            tr.append([(node,rnode),min_seg_thresh*2,D])

                    threshold.append([rs,delay,tr])
                results0[t]=threshold
        except:results0[t]=-1
    
        #2 risk_country
        try:
            if (not f2) and (not cflag2):
                nlrp,rsats=nlrp1,rsats1
                graph_opt=load_topo_opt(rsats,satPos)
                min_delay=nx.dijkstra_path_length(graph_opt,sd[0],sd[1])
                threshold=[min_delay]
                for enlarge in range(3):
                    delay=0
                    tr=[]
                    rs=[sd[0]]
                    rs.extend(DRSA(sd[0],sd[1],ds,dd,nlrp,enlarge))
                    rs.append(sd[1])

                    for ind in range(len(rs)-1):
                        D=nx.dijkstra_path_length(graph,rs[ind],rs[ind+1])
                        delay+=D#all_delay[rs[ind]][rs[ind+1]]
                        min_seg_thresh=1000
                        rnode=-1
                        if len(rsats)==0:
                            tr.append([])
                        else:
                            path=nx.dijkstra_path(graph,rs[ind],rs[ind+1])
                            for node in path:#all_path[rs[ind]][rs[ind+1]]:
                                for rsid in rsats:
                                    delays=nx.dijkstra_path_length(graph,rsid,node)#all_delay[rsid][node]
                                    if delays<min_seg_thresh:
                                        rnode=rsid
                                        min_seg_thresh=delays
                            tr.append([(node,rnode),min_seg_thresh*2,D])
                    threshold.append([rs,delay,tr])
                results1[t]=threshold
        except:results1[t]=-1
            
        
    def run_thread(sd_no,tstart,tend,risk_country_ls):
        for t in range(tstart,tend):
            if t%10==0:print(f'{sd_no}:{t}')
            sd_pre=[]
            rsats_pre=[[],[]]
            nlrp_pre=[[],[]]
            if t!=tstart:
                sd_pre=load_sd_sats(sd_no,t-1,constellation_name)
                rc0=risk_country_ls[0]
                nlrp0,rsats0=load_nlrp(t-1,rc0)
                rsats_pre[0]=rsats0
                nlrp_pre[0]=nlrp0
                rc0=risk_country_ls[1]
                nlrp0,rsats0=load_nlrp(t-1,rc0)
                rsats_pre[1]=rsats0
                nlrp_pre[1]=nlrp0
            starveri_main(sd_no,tstart,t,risk_country_ls,sd_pre,rsats_pre,nlrp_pre)
    step=lasting//thread_num if lasting%thread_num==0 else lasting//thread_num+1
    T=[]
    with concurrent.futures.ThreadPoolExecutor(max_workers=thread_num) as executor:
        for _ in range(thread_num):
            tstart=_*step
            tend=tstart+step if tstart+step<lasting else lasting
#             run_thread(sd_no,tstart,tend,risk_country_ls)
#         T.append(Thread(target=run_thread,args=(sd_no,tstart,tend,risk_country_ls)))
            T.append(executor.submit(run_thread,sd_no,tstart,tend,risk_country_ls))
    
# #     run_thread(sd_no,0,5,risk_country_ls)
#     for p in T:
#         p.start()
#     for p in T:
#         p.join()
    done, not_done = concurrent.futures.wait(T, return_when=concurrent.futures.ALL_COMPLETED)
    #run_thread(sd_no,0,1,risk_country_ls)
    
    #     T=[]
#     for ind in range(lasting):
#         if results0[ind]==[]:
#             start=ind
#             end=start+step if start+step<lasting else lasting
#             T.append(Thread(target=run_thread,args=(sd_no,start,end,risk_country_ls)))
#             ind+=step
#     for p in T:
#         p.start()
#     for p in T:
#         p.join()
#             with concurrent.futures.ThreadPoolExecutor(max_workers=thread_num) as executor:
#                 T.append(executor.submit(run_thread,sd_no,start,end,risk_country_ls))
#             ind+=step
#             done, not_done = concurrent.futures.wait(T, return_when=concurrent.futures.ALL_COMPLETED)
    with open(f'/data/chenwei/research3/data/StarVeri_ICNP/starveri/verify_results/{risk_country_ls[0]}_{constellation_name}/{sd_no}.csv','w')as f1:
        writer = csv.writer(f1)
        for info in results0:
            row=[]
            for ele in info:
                row.append(str(ele))
            writer.writerow(row)
    with open(f'/data/chenwei/research3/data/StarVeri_ICNP/starveri/verify_results/{risk_country_ls[1]}_{constellation_name}/{sd_no}.csv','w')as f1:
        writer = csv.writer(f1)
        for info in results1:
            row=[]
            for ele in info:
                row.append(str(ele))
            writer.writerow(row)
    print(sd_no,'done!')
def starveri_run_main(risk_country_ls,lasting,thread_num,constellation_name='Starlink1'):
    sd_no=0
    pool = multiprocessing.Pool(40)
    while sd_no<1:
        
#         if os.path.exists(f'/data/chenwei/research3/data/StarVeri_ICNP/starveri/verify_results/{risk_country_ls[0]}_{constellation_name}/{sd_no}.csv'):
#             sd_no+=1
#             continue
        #print(sd_no,'run')
        pool.apply_async(starveri_run,args=(sd_no,risk_country_ls,lasting,thread_num,constellation_name))
        sd_no+=1
    pool.close()
    pool.join()
#     starveri_run(0,risk_country_ls,lasting,thread_num,constellation_name)
#starveri_run_main(risk_country_ls=['Egypt'],lasting=6000,thread_num=20,constellation_name='Kuiper1')    
            
